In [163]:
from pathlib import Path 
import requests
import pandas as pd
import numpy as np
from io import StringIO
from simple_salesforce import Salesforce,api
import shutil
import datetime
from datetime import date


sf = Salesforce(
    username="jmartinez@careassistance.com",
    password="Jcmm010483",
    security_token="nc20DsgxOHWVR9OHZ78s7TRp",
)

In [164]:
# Recuperar empresas ID de salesforce desde Account
results = sf.query_all(
    """
  select Id, name, Holding__c ,Campa_a__c,Id_Empresa__c,Empleador__c,poliza__c  from account where ispersonaccount = false 
    """
)
Empresas_SF = pd.DataFrame(results["records"])
Empresas_SF = Empresas_SF[
    [
        "Id",
        "Name",
        "Holding__c",
        "Campa_a__c",
        "Id_Empresa__c",
        "Empleador__c",
        "poliza__c",
    ]
]
Empresas_SF.shape

(6746, 7)

In [165]:
# buscar empresas desde Programas_por_Empresas__c datos de empresas
results = sf.query_all(
    """
   select Id,Empresa__c, Empresa_Padre__c,Nombre_Empresa__c, Name  from Programas_por_Empresas__c 
    """
)
Programas_por_empresa_SF = pd.DataFrame(results["records"])

Programas_por_empresa_SF = Programas_por_empresa_SF[
    ["Id", "Empresa__c", "Empresa_Padre__c", "Nombre_Empresa__c", "Name"]
]
Programas_por_empresa_SF.shape

(6733, 5)

In [166]:
#Union de Programas Empresa y empresas
Programas_empresas = pd.merge(
    left=Empresas_SF,
    right=Programas_por_empresa_SF,
    how="left",
    left_on="Id",
    right_on="Empresa__c",
)
Programas_empresas = Programas_empresas.rename(
    columns={
        "Id_x": "Id Account Empresa",
        "Name_x": "Nombre Empresa",
        "Holding__c": "Holding",
        "Campa_a__c": "Campaña",
        "Empleador__c": "Empleador",
        "poliza__c": "Poliza",
        "Id_y": "Programas_por_Empresas__c",
        "Id_Empresa__c": "Codigo EMP",
        "Name_y": "Codigo A",
    }
)
Programas_empresas = Programas_empresas[
    [
        "Id Account Empresa",
        "Nombre Empresa",
        "Holding",
        "Campaña",
        "Empleador",
        "Poliza",
        "Programas_por_Empresas__c",
        "Codigo EMP",
        "Codigo A"
    ]
]

Programas_empresas.shape

(6752, 9)

fin empresas y comienzo de rescate de pacientes


In [167]:
#CArga de PAcientes
results = sf.query_all(
    """
    select Id, IdentificationId__pc,Name, Identification_Type__c, Ocupaci_n__c,  PersonTitle ,PersonDepartment,HealthCloudGA__Gender__pc,Phone  from Account where ispersonaccount = True
   """
)
Pacientes_SF = pd.DataFrame(results["records"])

Pacientes_SF = Pacientes_SF[
    [
        "Id",
        "IdentificationId__pc",
        "Name",
        "Identification_Type__c",
        "Ocupaci_n__c",
        "PersonTitle",
        "PersonDepartment",
        "HealthCloudGA__Gender__pc",
        'Phone'
        
    ]
]
Pacientes_SF = Pacientes_SF.rename(
    columns={
        "Id": "Id Account Paciente",
        "IdentificationId__pc": "Rut",
        "Name": "Nombre",
        "Ocupaci_n__c": "Ocupacion",
        "PersonTitle": "Cargo",
        "PersonDepartment": "Departamento",
        "HealthCloudGA__Gender__pc": "Sexo",
        "Phone": "Telefono",
        "Identification_Type__c": "Tipo de Documento",
    }
)
Pacientes_SF.shape

(1583849, 9)

Aca se comienzan a unir las relaciones de pacientes y empresas ya creadas.

In [168]:
#buscar empresas desde Paciente_Programas__c relaciones de empresas con pacientes
results = sf.query_all(
    """
    select Id,Paciente__c, Programas_por_Empresas__c,padre__c from Paciente_Programas__c  
    """
)
Paciente_Programas_SF = pd.DataFrame(results["records"])

Paciente_Programas_SF = Paciente_Programas_SF[
    ["Id", "Paciente__c", "Programas_por_Empresas__c", "padre__c"]
]
Paciente_Programas_SF = Paciente_Programas_SF.rename(
    columns={
        "Id": "Id Pacientes_Programas__c",
        "Paciente__c": "Id Account",
        "Programas_por_Empresas__c": "Programas_por_Empresas__c",
        "padre__c": "Nombre Campaña"        
    }
)
Paciente_Programas_SF.shape

(1130538, 4)

In [169]:
#Union de Pacientes Programas y Programas por empresa
empresas_unidas = pd.merge(
    left=Paciente_Programas_SF,
    right=Programas_empresas,
    how="left",
    left_on="Programas_por_Empresas__c",
    right_on="Programas_por_Empresas__c", 
)
empresas_unidas.shape

(1130538, 12)

In [110]:
empresas_unidas.head()

,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
0,a1Y8c00000C5asnEAB,0018c00002amLbpAAE,a1W8c000007vDPvEAM,MetLife,0018c00002axQaeAAE,Serv. De Comerc. Y Ases. Callbusiness SPA,DBCH - CNSTD 2019,MetLife,Serv. De Comerc. Y Ases. Callbusiness SPA,340016490,EMP6008,A-0173
1,a1Y8c00000C5asoEAB,0018c00002amLbqAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
2,a1Y8c00000C5aspEAB,0018c00002amLbrAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
3,a1Y8c00000C5asqEAB,0018c00002amLbsAAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353
4,a1Y8c00000C5asrEAB,0018c00002amLc2AAE,a1W8c000007vDSpEAM,MetLife,0018c00002axQecAAE,Clínica Alemana,Clínica Alemana,MetLife,Clínica Alemana,340006119,EMP6274,A-0353


In [9]:
#pacientes_empresas22 = pd.merge(
    #left=Pacientes_SF,
    #right=empresas_unidas,
    #how="left",
    #left_on="Id Account Paciente",
    #right_on="Id Account", 
#)
#pacientes_empresas22 = pacientes_empresas22.drop_duplicates(['Rut'], keep='last')
#pacientes_empresas22 = pacientes_empresas22[
    #["Rut", "Nombre", "Nombre Campaña", "Nombre Empresa"]
#]


In [10]:
#pacientes_empresas22.shape

In [11]:
#apprix_1 = pacientes_empresas22.iloc[:848826,:]

#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts_habilitados1.xlsx",index=False)


In [12]:
#apprix_2 = pacientes_empresas22.iloc[848827:,:]#
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts_habilitados2.xlsx",index=False)

In [13]:
#pacientes_empresas22.head()

Carga de Archivo carga

In [151]:
nuevos = ("F:/Empresas SF/reportes/carga.xlsx")
campaña ='Lipigas'
Pais = 'Chile'




Pacientes_Archivo = pd.read_excel(nuevos)
# OJO se copia rut1 a rut le saca el guoion
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut1"].str.replace("-", "")

Pacientes_Archivo.head()                                       

,rut15,Unnamed: 1,nombres,apellidos,nacimiento,rut1,rut,RutEstaoNo,Id Account Paciente,Rut,Nombre
0,17587132-2,Rolando Jorge Chiguay Chaura,Rolando Jorge,Chiguay Chaura,32964,17587132-2,175871322,True,0018c00002eUTI3AAO,175871322.0,Rolando Chiguay Chaura
1,12891306-8,Ruben Ismale Curin Antillanca,Ruben Ismale,Curin Antillanca,27640,12891306-8,128913068,True,0018c00002cWKMAAA4,128913068.0,Ruben Ismael Curin Antillanca
2,18168131-4,Lucas Gabriel Meza Maguire,Lucas Gabriel,Meza Maguire,33820,18168131-4,181681314,False,NaN,NaN,NaN
3,12042664-8,Roberto Andrés Valenzuela Díaz,Roberto Andrés,Valenzuela Díaz,27533,12042664-8,120426648,True,0018c00002anyfuAAA,120426648.0,Valenzuela Diaz Roberto
4,14363593-7,Johanna Andrea Coello Villalobos,Johanna Andrea,Coello Villalobos,29729,14363593-7,143635937,True,0018c00002eU8oPAAS,143635937.0,Johanna Andrea Coello Villalobos


In [152]:
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")

# aca se trabaja con el nuevo rut desde rut1 ojooooooo
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.upper()
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].astype(str)
Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.strip()
Pacientes_Archivo.head() 

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6212\1036101616.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["rut"] = Pacientes_Archivo["rut"].str.replace(".", "")


,rut15,Unnamed: 1,nombres,apellidos,nacimiento,rut1,rut,RutEstaoNo,Id Account Paciente,Rut,Nombre
0,17587132-2,Rolando Jorge Chiguay Chaura,Rolando Jorge,Chiguay Chaura,32964,17587132-2,175871322,True,0018c00002eUTI3AAO,175871322.0,Rolando Chiguay Chaura
1,12891306-8,Ruben Ismale Curin Antillanca,Ruben Ismale,Curin Antillanca,27640,12891306-8,128913068,True,0018c00002cWKMAAA4,128913068.0,Ruben Ismael Curin Antillanca
2,18168131-4,Lucas Gabriel Meza Maguire,Lucas Gabriel,Meza Maguire,33820,18168131-4,181681314,False,NaN,NaN,NaN
3,12042664-8,Roberto Andrés Valenzuela Díaz,Roberto Andrés,Valenzuela Díaz,27533,12042664-8,120426648,True,0018c00002anyfuAAA,120426648.0,Valenzuela Diaz Roberto
4,14363593-7,Johanna Andrea Coello Villalobos,Johanna Andrea,Coello Villalobos,29729,14363593-7,143635937,True,0018c00002eU8oPAAS,143635937.0,Johanna Andrea Coello Villalobos


In [153]:
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(",", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace("-", "")
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.title()
Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.strip()
#Pacientes_Archivo["empresa"] = Pacientes_Archivo["empresa"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(",", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace("-", "")
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.title()
Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.upper()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].astype(str)
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.strip()
Pacientes_SF["Rut"] = Pacientes_SF["Rut"].str.replace(
    "-", ""
)


Pacientes_Archivo["RutEstaoNo"] = Pacientes_Archivo["rut"].isin(
    Pacientes_SF["Rut"])

C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6212\2096211457.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["nombres"] = Pacientes_Archivo["nombres"].str.replace(".", "")
C:\Users\jmartinez\AppData\Local\Temp\ipykernel_6212\2096211457.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  Pacientes_Archivo["apellidos"] = Pacientes_Archivo["apellidos"].str.replace(".", "")


In [154]:
#aca a los pacientes del archivo se les asigna sus datos en SF
Pacientes_con_id = pd.merge(
    left=Pacientes_Archivo,
    right=Pacientes_SF,
    how="left",
    left_on="rut",
    right_on="Rut",
)
Pacientes_con_id.shape

(28, 20)

In [155]:
Pacientes_con_id.head()

,rut15,Unnamed: 1,nombres,apellidos,nacimiento,rut1,rut,RutEstaoNo,Id Account Paciente_x,Rut_x,Nombre_x,Id Account Paciente_y,Rut_y,Nombre_y,Tipo de Documento,Ocupacion,Cargo,Departamento,Sexo,Telefono
0,17587132-2,Rolando Jorge Chiguay Chaura,Rolando Jorge,Chiguay Chaura,32964,17587132-2,175871322,True,0018c00002eUTI3AAO,175871322.0,Rolando Chiguay Chaura,0018c00002eUTI3AAO,175871322,Rolando Chiguay Chaura,RUT,None,None,None,Hombre,None
1,12891306-8,Ruben Ismale Curin Antillanca,Ruben Ismale,Curin Antillanca,27640,12891306-8,128913068,True,0018c00002cWKMAAA4,128913068.0,Ruben Ismael Curin Antillanca,0018c00002cWKMAAA4,128913068,Ruben Ismael Curin Antillanca,RUT,None,None,None,Hombre,None
2,18168131-4,Lucas Gabriel Meza Maguire,Lucas Gabriel,Meza Maguire,33820,18168131-4,181681314,True,NaN,NaN,NaN,0018c00002gkuqtAAA,181681314,Lucas Gabriel Meza Maguire,RUT,None,None,None,None,None
3,12042664-8,Roberto Andrés Valenzuela Díaz,Roberto Andrés,Valenzuela Díaz,27533,12042664-8,120426648,True,0018c00002anyfuAAA,120426648.0,Valenzuela Diaz Roberto,0018c00002anyfuAAA,120426648,Valenzuela Diaz Roberto,RUT,None,None,None,Hombre,None
4,14363593-7,Johanna Andrea Coello Villalobos,Johanna Andrea,Coello Villalobos,29729,14363593-7,143635937,True,0018c00002eU8oPAAS,143635937.0,Johanna Andrea Coello Villalobos,0018c00002eU8oPAAS,143635937,Johanna Andrea Coello Villalobos,RUT,None,None,None,Mujer,None


In [156]:
abraham = Pacientes_con_id[["rut1", "rut", "Id Account Paciente_x"]]
abraham = abraham.rename(
    columns={
        "rut1": "rut1",
        "rut": "Rut",
        "Id Account Paciente_x": "Paciente__c",
    }
)
abraham.to_excel("F:/Empresas SF/reportes/abraham.xlsx", index=False)

In [157]:
Programas_empresas.to_excel("F:/Empresas SF/reportes/empresas.xlsx",index=False)

In [158]:
# Final_pacientes_y_empresa = Pacientes_con_ID
Subir_pacientes = pd.DataFrame()
Subir_pacientes["RutEstaoNo"] = Pacientes_con_id["RutEstaoNo"]
# agrupar = empresasqfaltan[empresasqfaltan['si']==False]


if ("nombres" in Pacientes_con_id.columns) == True:
    Subir_pacientes["FirstName"] = Pacientes_con_id["nombres"]
if ("apellidos" in Pacientes_con_id.columns) == True:
    Subir_pacientes["LastName"] = Pacientes_con_id["apellidos"]
if ("rut" in Pacientes_con_id.columns) == True:
    Subir_pacientes["IdentificationId__pc"] = Pacientes_con_id["rut"]
if ("email" in Pacientes_con_id.columns) == True:
    # Pacientes_con_id["email"] = Pacientes_con_id["email"].astype(str).str.lower().replace("nan","")
    Subir_pacientes["PersonEmail"] = (
        Pacientes_con_id["email"].astype(str).str.lower().replace("nan", "")
    )
if ("nacimiento" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonBirthdate"] = Pacientes_con_id["nacimiento"]
if ("sexo" in Pacientes_con_id.columns) == True:
    Sexo = {
        "M": "Hombre",
        "m": "Hombre",
        "Masculino": "Hombre",
        "Femenino": "Mujer",
        "": "Prefiero no responder",
        "HOMBRE": "Hombre",
        "MUJER": "Mujer",
        "F": "Mujer",
        "O": "O",
        "Ambos": "Prefiero no responder",
        "N": "Prefiero no responder",
        "Otro": "Prefiero no responder",
        "n": "Prefiero no responder",
        "N": "Prefiero no responder",
        "FEMENINO": "Mujer",
        "f": "Mujer",
        "MASCULINO": "Hombre",
        "m": "Hombre",
        "M ": "Hombre",
        "F ": "Mujer",
        'Varón':'Hombre'      
        
    }
    Pacientes_con_id.replace({"sexo": Sexo}, inplace=True)
    Subir_pacientes["HealthCloudGA__Gender__pc"] = Pacientes_con_id["sexo"]
if ("direccion" in Pacientes_con_id.columns) == True:
    Subir_pacientes["BillingStreet"] = (
        Pacientes_con_id["direccion"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingStreet"] = Final_pacientes_y_empresa["direccion"]
if ("comuna" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
    Subir_pacientes["BillingState"] = (
        Pacientes_con_id["comuna"].str.title().replace("NaN", "")
    )
    # Subir_pacientes["BillingState"] = Final_pacientes_y_empresa["comuna"]
if ("ciudad" in Pacientes_con_id.columns) == True:
    # Subir_pacientes["BillingCity"] = Final_pacientes_y_empresa["ciudad"]
    Subir_pacientes["BillingCity"] = (
        Pacientes_con_id["ciudad"].str.title().replace("NaN", "")
    )

if ("cargo" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonTitle"] = Pacientes_con_id["cargo"].str.title()

if ("division" in Pacientes_con_id.columns) == True:
    Subir_pacientes["PersonDepartment"] = Pacientes_con_id["division"].str.title()

if ("celular" in Pacientes_con_id.columns) == True:
    Subir_pacientes["Phone"] = Pacientes_con_id["celular"]

Subir_pacientes["RecordTypeId"] = "0128c000002Byp4AAC"




if Pais == 'Chile':        
    Subir_pacientes["BillingCountry"] = "Chile"
    Subir_pacientes["Identification_Type__c"] = "RUT"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000005dAtnEAE'
else:    
    Subir_pacientes["BillingCountry"] = "Mexico"
    Subir_pacientes["Identification_Type__c"] = "CURP"
    Subir_pacientes['Pa_s_del_Paciente__c'] = 'a1U8c000004cvS7EAI'


Subir_pacientes = Subir_pacientes[Subir_pacientes["RutEstaoNo"] == False]


# Subir_pacientes.PersonEmail = Subir_pacientes.PersonEmail.fillna('')
Subir_pacientes.to_excel("F:/Empresas SF/reportes/account.xlsx", index=False)
Subir_pacientes.head()

,RutEstaoNo,FirstName,LastName,IdentificationId__pc,PersonBirthdate,RecordTypeId,BillingCountry,Identification_Type__c,Pa_s_del_Paciente__c


In [159]:
    
clinica_virtual = pd.DataFrame()    
clinica_virtual['rut'] = Pacientes_con_id["rut1"].str.upper()
clinica_virtual['firstName'] = Pacientes_con_id['nombres']
clinica_virtual['lastName'] = Pacientes_con_id['apellidos']
clinica_virtual['accountId'] = campaña



Codigos_Clinica_Virtual = {
'BCI':'1',
'Farmacias Ahumada':'2',
'MetLife':'3',
'Vida Security':'4',
'Willis':'5',
'Entel':'6',
'Testing':'7',
'3SP':'8',
'HomeCare':'9',
'Pedro de Valdivia':'10',
'Colaboradores':'11',
'MineraAbra':'12',
'CMPC':'13',
'Lipigas':'14',
'Asismed':'15',
'Siempre Perú':'16',
'Farmacias Cruz Verde':'17',
'WFS Food Services':'18',
'VidaCel':'19',
'Dermik':'20',
'EY':'21',
'IND':'22',
'Sindicato Walmart':'23',
'Instruvalve':'24',
'VidaCel Colaboradores':'25',
'Socofar':'26',
'Milan Fajanovic':'27',
'THBseguros':'28',
'Pesco':'29',
'Burgos - Consorcio':'30',
'COFAMA-SILENTIUM':'31',
'KAUFMANN':'32',
'QuePlan':'33',
'Zurich':'34',
'Colaboradores Siempre Chile':'35',
'Talo':'36',
'TRANSVIP':'37',
'Vida Security Colaboradores':'38',
'FORUS':'39',
'SURA':'40',
'Southbrige':'41',
'Construmart':'42',
'BICE Vida':'43'

}

clinica_virtual.replace({"accountId": Codigos_Clinica_Virtual}, inplace=True)
clinica_virtual.to_excel("F:/Empresas SF/reportes/Clinica_Virtual.xlsx", index=False)

In [160]:
campaña_seleccionada = empresas_unidas[empresas_unidas["Campaña"] == campaña]
campaña_seleccionada.head()


,Id Pacientes_Programas__c,Id Account,Programas_por_Empresas__c,Nombre Campaña,Id Account Empresa,Nombre Empresa,Holding,Campaña,Empleador,Poliza,Codigo EMP,Codigo A
159924,a1Y8c00000C6VZMEA3,0018c00002amQQvAAM,a1W8c0000083iWHEAY,Lipigas,0018c00002cXJTRAA4,Empresas Lipigas S.A.,Sin Holding Asociado,Lipigas,Empresas Lipigas S.A.,Sin Póliza Asociada,EMP7310,A-0695
159925,a1Y8c00000C6VZNEA3,0018c00002amQQPAA2,a1W8c0000083iWHEAY,Lipigas,0018c00002cXJTRAA4,Empresas Lipigas S.A.,Sin Holding Asociado,Lipigas,Empresas Lipigas S.A.,Sin Póliza Asociada,EMP7310,A-0695
159926,a1Y8c00000C6VZPEA3,0018c00002amVK6AAM,a1W8c0000083iWHEAY,Lipigas,0018c00002cXJTRAA4,Empresas Lipigas S.A.,Sin Holding Asociado,Lipigas,Empresas Lipigas S.A.,Sin Póliza Asociada,EMP7310,A-0695
159927,a1Y8c00000C6VZQEA3,0018c00002amVK8AAM,a1W8c0000083iWHEAY,Lipigas,0018c00002cXJTRAA4,Empresas Lipigas S.A.,Sin Holding Asociado,Lipigas,Empresas Lipigas S.A.,Sin Póliza Asociada,EMP7310,A-0695
159928,a1Y8c00000C6VZREA3,0018c00002amPXPAA2,a1W8c0000083iWHEAY,Lipigas,0018c00002cXJTRAA4,Empresas Lipigas S.A.,Sin Holding Asociado,Lipigas,Empresas Lipigas S.A.,Sin Póliza Asociada,EMP7310,A-0695


In [161]:
campaña_seleccionada.shape

(172, 12)

In [162]:
if campaña != "MetLife":
    pacientes_en_campaña = pd.merge(
        left=campaña_seleccionada,
        right=Pacientes_con_id,
        how="left",
        left_on="Id Account",
        right_on="Id Account Paciente",
    )
    pacientes_en_campaña["Paciente__c"] = "Id Pacientes"
    pacientes_en_campaña["Programas_por_Empresas__c"] = "Id Programas Por empresa"
    pacientes_en_campaña.to_excel(
        "F:/Empresas SF/reportes/campaña_empresa.xlsx", index=False
    )
    campaña_en_pacientes = pd.merge(
        left=Pacientes_con_id,
        right=campaña_seleccionada,
        how="left",
        left_on="Id Account Paciente",
        right_on="Id Account",
    )
    campaña_en_pacientes.to_excel(
        "F:/Empresas SF/reportes/archivo_campaña.xlsx", index=False
    )

KeyError: 'Id Account Paciente'

In [ ]:

#apprix_1 = Pacientes_SF.iloc[:790552,:]
#apprix_2 = Pacientes_SF.iloc[790553:,:]
#apprix_1.to_excel("F:/Empresas SF/reportes/Ruts1.xlsx",index=False)
#apprix_2.to_excel("F:/Empresas SF/reportes/Ruts2.xlsx",index=False)
